In [ ]:
import warnings
import zipfile
import random
import os
import requests
import shutil
from justhtml import JustHTML
from shutil import unpack_archive
from tempfile import TemporaryDirectory

def load_page(url):
    r = requests.get(url)
    r.raise_for_status()
    return r.text

def download_file(url, dir: str | os.PathLike, ext_blacklist=None):
    r = requests.get(url, stream=True)
    r.raise_for_status()

    fname = r.headers.get("Content-Disposition")
    if fname is None: fname = str(random.random())

    if ext_blacklist is not None:
        if isinstance(ext_blacklist, str): ext_blacklist = (ext_blacklist, )
        if fname.strip().lower().endswith(tuple(ext_blacklist)):
            return None

    with open(os.path.join(dir, fname), 'wb') as f:
        shutil.copyfileobj(r.raw, f) # type:ignore

    return os.path.join(dir, fname)

def get_topics(html: JustHTML) -> set[str]:
    topics = set()
    for el in html.query("div > a"):
        if "href" in el.attrs:
            href = el.attrs["href"]
            if href.startswith("./viewtopic.php?t="):
                topics.add(f"https://forum.generally-racers.com/{href[2:]}")
    return topics

def get_download_links(html: JustHTML):
    links = set()
    for el in html.query("dt > a"):
        if "href" in el.attrs:
            href = el.attrs["href"]
            if href.startswith("./download"):
                links.add(f"https://forum.generally-racers.com/{href[2:]}")
    return links

def get_current_page_number(html: JustHTML):
    for el in html.query("ul > li"):
        if el.attrs.get("class",  None) == "active":
            return int(el.children[0].to_text())
    return 1

def get_next_page_url(html: JustHTML):
    page_number = get_current_page_number(html)

    for el in html.query("ul > li > a"):
        if (el.attrs.get("class",  None) == "button") and ("href" in el.attrs):
            try:
                n = int(el.to_text())
            except (ValueError, TypeError):
                continue

            if n == page_number + 1:
                href = el.attrs["href"]
                return f"https://forum.generally-racers.com/{href[2:]}"

    return None


def unpack(file, dir):
    try:
        shutil.unpack_archive(file, dir)
    except Exception:
        with zipfile.ZipFile(file, "r") as zip_ref:
            zip_ref.extractall(dir)

def get_car_trk_files(file: str | os.PathLike, download_path: str | os.PathLike) -> None:
    if str(file).lower().strip().endswith((".car", ".trk")):
        print(f"SAVING {os.path.basename(file)}")
        if os.path.basename(file) in os.listdir(download_path):
            warnings.warn(f"file {os.path.basename(file)} already exists...")
        shutil.move(file, os.path.join(download_path, os.path.basename(file)))
    else:
        try:
            with TemporaryDirectory() as tmpdir:
                unpack(file, tmpdir)
                for root, dirs, files in os.walk(tmpdir):
                    for f in files:
                        get_car_trk_files(os.path.join(root, f), download_path)
        except Exception:
            pass

_BLACKLIST = (".jpeg", ".jpg", ".png", ".gif", ".mp4", ".mov", ".mp3", ".wav", ".tiff")
class GeneRallyCarDownloader:
    def __init__(self, download_path: str | os.PathLike):
        self.download_urls: set[str] = set()
        self.visited_pages: set[tuple[str, int]] = set()
        self.download_path = download_path

    def scrap_page(self, html: JustHTML):
        """downloads all files and returns url of next page or none"""
        dl_links = get_download_links(html)
        for dl_link in dl_links:
            if "&mode=view" in dl_link: continue
            print(f"DOWNLOADING {dl_link}")
            with TemporaryDirectory() as tmpdir:
                path = download_file(dl_link, tmpdir, _BLACKLIST)
                if path is not None:
                    get_car_trk_files(path, self.download_path)

    def run(self, root = "https://forum.generally-racers.com/viewforum.php?f=7"):
        current_page_url = root
        current_html = JustHTML(load_page(current_page_url))
        while True:
            topics = get_topics(current_html)
            for topic_url in topics:
                print(f"LOADING TOPIC {topic_url}")
                topic_html = JustHTML(load_page(topic_url))
                while True:
                    self.scrap_page(topic_html)
                    next_url = get_next_page_url(topic_html)
                    print(f"LOADING PAGE {next_url}")
                    if next_url is None: break
                    topic_html = JustHTML(load_page(next_url))

            current_page_url = get_next_page_url(current_html)
            if current_page_url is None: break
            current_html = JustHTML(load_page(current_page_url))

scrapper = GeneRallyCarDownloader("/var/mnt/issd/files 2/programming/experiments/generally scrapper/downloads")
scrapper.run()

LOADING TOPIC https://forum.generally-racers.com/viewtopic.php?t=5026&sid=f07ac1d0744c78c6a1644a48cb45b836
DOWNLOADING https://forum.generally-racers.com/download/file.php?id=24011&sid=f07ac1d0744c78c6a1644a48cb45b836
SAVING MCarGT16-V3.car
LOADING PAGE None
LOADING TOPIC https://forum.generally-racers.com/viewtopic.php?t=5557&sid=f07ac1d0744c78c6a1644a48cb45b836
DOWNLOADING https://forum.generally-racers.com/download/file.php?id=25370&sid=f07ac1d0744c78c6a1644a48cb45b836
SAVING GR World map by Herbal.car
DOWNLOADING https://forum.generally-racers.com/download/file.php?id=25369&sid=f07ac1d0744c78c6a1644a48cb45b836&mode=view
LOADING PAGE None
LOADING TOPIC https://forum.generally-racers.com/viewtopic.php?t=5039&sid=f07ac1d0744c78c6a1644a48cb45b836
DOWNLOADING https://forum.generally-racers.com/download/file.php?id=24122&sid=f07ac1d0744c78c6a1644a48cb45b836
SAVING DirtOvalSprintCar.car
LOADING PAGE None
LOADING TOPIC https://forum.generally-racers.com/viewtopic.php?t=5&sid=f07ac1d0744c78

/tmp/ipykernel_272039/4094920146.py:85: UserWarning: file [SUTINEN] Silvia pack addon.car already exists...
  warnings.warn(f"file {os.path.basename(file)} already exists...")


DOWNLOADING https://forum.generally-racers.com/download/file.php?id=2534&sid=f07ac1d0744c78c6a1644a48cb45b836&mode=view
DOWNLOADING https://forum.generally-racers.com/download/file.php?id=2381&sid=f07ac1d0744c78c6a1644a48cb45b836&mode=view
LOADING PAGE https://forum.generally-racers.com/viewtopic.php?t=3&sid=f07ac1d0744c78c6a1644a48cb45b836&start=225
LOADING PAGE https://forum.generally-racers.com/viewtopic.php?t=3&sid=f07ac1d0744c78c6a1644a48cb45b836&start=250
DOWNLOADING https://forum.generally-racers.com/download/file.php?id=2706&sid=f07ac1d0744c78c6a1644a48cb45b836&mode=view
DOWNLOADING https://forum.generally-racers.com/download/file.php?id=2620&sid=f07ac1d0744c78c6a1644a48cb45b836&mode=view
DOWNLOADING https://forum.generally-racers.com/download/file.php?id=2703&sid=f07ac1d0744c78c6a1644a48cb45b836&mode=view
DOWNLOADING https://forum.generally-racers.com/download/file.php?id=2686&sid=f07ac1d0744c78c6a1644a48cb45b836&mode=view
DOWNLOADING https://forum.generally-racers.com/downloa